# Imports

In [115]:
from kaggle_environments import evaluate, make, utils
import random
import numpy as np

### General Helper Functions

These functions are provided by kaggle

In [116]:
def get_win_percentages(agent1, agent2, n_rounds=100):
    # Use default Connect Four setup
    config = {'rows': 6, 'columns': 7, 'inarow': 4}
    # Agent 1 goes first (roughly) half the time          
    outcomes = evaluate("connectx", [agent1, agent2], config, [], n_rounds//2)
    # Agent 2 goes first (roughly) half the time      
    outcomes += [[b,a] for [a,b] in evaluate("connectx", [agent2, agent1], config, [], n_rounds-n_rounds//2)]
    Agent1 = agent1 if isinstance(agent1, str) else agent1.__name__
    Agent2 = agent2 if isinstance(agent2, str) else agent2.__name__
    print(f'{Agent1} Win Percentage:', np.round(outcomes.count([1,-1])/len(outcomes), 2))
    print(f'{Agent2} Win Percentage:', np.round(outcomes.count([-1,1])/len(outcomes), 2))
    print(f'Number of Invalid Plays by {Agent1}:', outcomes.count([None, 0]))
    print(f'Number of Invalid Plays by {Agent2}:', outcomes.count([0, None]))

In [117]:
# Gets board at next step if agent drops piece in selected column
def drop_piece(grid, col, piece, config):
    next_grid = grid.copy()
    for row in range(config.rows-1, -1, -1):
        if next_grid[row][col] == 0:
            break
    next_grid[row][col] = piece
    return next_grid

# Returns True if dropping piece in column results in game win
def check_winning_move(grid, config, col, piece):
    # Convert the board to a 2D grid
    next_grid = drop_piece(grid, col, piece, config)
    # horizontal
    for row in range(config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(next_grid[row,col:col+config.inarow])
            if window.count(piece) == config.inarow:
                return True
    # vertical
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns):
            window = list(next_grid[row:row+config.inarow,col])
            if window.count(piece) == config.inarow:
                return True
    # positive diagonal
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns-(config.inarow-1)):
            window = list(next_grid[range(row, row+config.inarow), range(col, col+config.inarow)])
            if window.count(piece) == config.inarow:
                return True
    # negative diagonal
    for row in range(config.inarow-1, config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(next_grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
            if window.count(piece) == config.inarow:
                return True
    return False

### Agents without heuristics

First attempts at creating agents. They don't play based on a heuristic function but play random moves, unless a move is winning or an opponent's winning move can be blocked

In [118]:
def manual_agent(obs, config):
    
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    valid_moves = [col for col in range(config.columns) if grid[0][col] == 0]
    
    for move in valid_moves:
        if check_winning_move(grid, config, move, obs.mark):
            return move
        
    for move in valid_moves:
        if check_winning_move(grid, config, move, 3 - obs.mark):
            return move
    
    return random.choice(valid_moves)

In [119]:
def manual_agent_improved(obs, config):
    
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    valid_moves = [col for col in range(config.columns) if grid[0][col] == 0]
    
    for move in valid_moves:
        if check_winning_move(grid, config, move, obs.mark):
            return move
        
    for move in valid_moves:
        if check_winning_move(grid, config, move, 3 - obs.mark):
            return move
        
    not_losing_moves = valid_moves[:]   
    
    for move in valid_moves:
        next_grid = drop_piece(grid, move, obs.mark, config)
        opponent_valid_moves = [col for col in range(config.columns) if next_grid[0][col] == 0]
        for opponent_move in opponent_valid_moves:
            if check_winning_move(next_grid, config, opponent_move, 3 - obs.mark):
                not_losing_moves.remove(move)
                break
                
    if not_losing_moves:
        return random.choice(not_losing_moves)
    
    return random.choice(valid_moves)

### Heuristic helper functions

Helper functions used to create heuristic functions which evaluate how good a position is for the agent

In [120]:
# Calculates score if agent drops piece in selected column based on the heuristic function provided as argument
def score_single_move(heuristic, grid, col, mark, config):
    next_grid = drop_piece(grid, col, mark, config)
    score = heuristic(next_grid, mark, config)
    return score

# Helper function for get_heuristic: checks if window has num_discs of consecutive pieces and the remaining positions are unfilled
def check_window(window, num_discs, piece, config):
    return (window.count(piece) == num_discs and window.count(0) == config.inarow-num_discs)
    
# Helper function for get_heuristic: counts number of windows satisfying specified heuristic conditions
def count_windows(grid, num_discs, piece, config):
    num_windows = 0
    # horizontal
    for row in range(config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[row, col:col+config.inarow])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    # vertical
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns):
            window = list(grid[row:row+config.inarow, col])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    # positive diagonal
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    # negative diagonal
    for row in range(config.inarow-1, config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    return num_windows


### Heuristics 

In [121]:
def heuristic_1_grid(grid, mark, config):
    num_threes = count_windows(grid, 3, mark, config)
    num_fours = count_windows(grid, 4, mark, config)
    num_threes_opp = count_windows(grid, 3, mark%2+1, config)
    score = num_threes - 5*num_threes_opp + 100*num_fours
    return score

def heuristic_2_window(window, piece, config):
    # for fast heuristic
    window_score = 1000*check_window(window, 4, piece, config)
    window_score += 10*check_window(window, 3, piece, config)
    window_score += 2*check_window(window, 2, piece, config)
    window_score -= check_window(window, 2, 3-piece, config) 
    window_score -= 100*check_window(window, 3, 3-piece, config)
    return window_score

def change_in_heuristic(grid, heuristic, col, piece, config, maximizingPlayer):
    change = 0
    for row in range(config.rows-1, -1, -1):
        if grid[row][col] == 0:
            break  
    # horizontal
    for i in range(config.inarow):
        if col - i >= 0 and col + config.inarow - i <= config.columns:
            old_window = list(grid[row, range(col-i, col+config.inarow-i)]) 
            new_window = old_window[:]
            new_window[i] = piece
            if maximizingPlayer:
                change += heuristic(new_window, piece, config) - heuristic(old_window, piece, config)
            else:
                change += heuristic(new_window, 3-piece, config) - heuristic(old_window, 3-piece, config)
     
    # vertical      
    for i in range(config.inarow):
        if row - i >= 0 and row - i + config.inarow <= config.rows:
            old_window = list(grid[range(row - i, row - i + config.inarow ) , col]) 
            new_window = old_window[:]
            new_window[i] = piece
            if maximizingPlayer:
                change += heuristic(new_window, piece, config) - heuristic(old_window, piece, config)
            else:
                change += heuristic(new_window, 3-piece, config) - heuristic(old_window, 3-piece, config)
     
    # positive diagonal
    for i in range(config.inarow):
        if col >= i and col <= config.columns-config.inarow+i and row >= config.inarow-1-i and row <= config.rows-1-i:
            old_window = list(grid[range(row + i, row + i - config.inarow, -1), range(col-i, col+config.inarow-i)])
            new_window = old_window[:]   
            new_window[i] = piece
            if maximizingPlayer:
                change += heuristic(new_window, piece, config) - heuristic(old_window, piece, config)
            else:
                change += heuristic(new_window, 3-piece, config) - heuristic(old_window, 3-piece, config)
     
    # negative diagonal
    for i in range(config.inarow):
        if col-i >= 0 and col+config.inarow-i <= config.columns and row-i >= 0 and row-i+config.inarow <= config.rows:
            old_window = list(grid[range(row - i, row - i + config.inarow ) , range(col-i, col-i + config.inarow)])
            new_window = old_window[:] 
            new_window[i] = piece   
            if maximizingPlayer:
                change += heuristic(new_window, piece, config) - heuristic(old_window, piece, config)
            else:
                change += heuristic(new_window, 3-piece, config) - heuristic(old_window, 3-piece, config)
     
    return change


def heuristic_3_window(window, piece, config):
    # for depth 2 minimax
    window_score = 500*check_window(window, 4, piece, config)
    window_score += 10*check_window(window, 3, piece, config)
    window_score += 2*check_window(window, 2, piece, config)
    window_score -= check_window(window, 2, 3-piece, config) 
    window_score -= 10*check_window(window, 3, 3-piece, config)
    window_score -= 1000*check_window(window,4, 3-piece, config)
    return window_score

### Heuristic Agents

In [122]:
def base_heuristic_agent(obs, config):
    # Get list of valid moves
    valid_moves = [c for c in range(config.columns) if obs.board[c] == 0]
    # Convert the board to a 2D grid
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    # Use the heuristic to assign a score to each possible board in the next turn
    scores = dict(zip(valid_moves, [score_single_move(heuristic_1_grid, grid, col, obs.mark, config) for col in valid_moves]))
    # Get a list of columns (moves) that maximize the heuristic
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]
    # Select at random from the maximizing columns
    return random.choice(max_cols)

def fast_heuristic_agent(obs, config):
    
    valid_moves = [c for c in range(config.columns) if obs.board[c] == 0]
    
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    
    score_differences = dict(zip(valid_moves, [change_in_heuristic(grid, heuristic_2_window , col, obs.mark, config, True) for col in valid_moves]))
    
    max_cols = [key for key in score_differences.keys() if score_differences[key] == max(score_differences.values())]

    return random.choice(max_cols)

### Minimax Helper Functions

In [123]:
# Helper function for minimax: checks if agent or opponent has four in a row in the window
def is_terminal_window(window, config):
    return window.count(1) == config.inarow or window.count(2) == config.inarow

# Helper function for minimax: checks if game has ended
def is_terminal_node(grid, config):
    # Check for draw 
    if list(grid[0, :]).count(0) == 0:
        return True
    # Check for win: horizontal, vertical, or diagonal
    # horizontal 
    for row in range(config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[row, col:col+config.inarow])
            if is_terminal_window(window, config):
                return True
    # vertical
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns):
            window = list(grid[row:row+config.inarow, col])
            if is_terminal_window(window, config):
                return True
    # positive diagonal
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
            if is_terminal_window(window, config):
                return True
    # negative diagonal
    for row in range(config.inarow-1, config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
            if is_terminal_window(window, config):
                return True
    return False

###  Naive minimax 

In [124]:
def minimax_naive(node, depth, maximizingPlayer, mark, config):
    is_terminal = is_terminal_node(node, config)
    valid_moves = [c for c in range(config.columns) if node[0][c] == 0]
    if depth == 0 or is_terminal:
        return heuristic_1_grid(node, mark, config)
    if maximizingPlayer:
        value = -np.Inf
        for col in valid_moves:
            child = drop_piece(node, col, mark, config)
            value = max(value, minimax_naive(child, depth-1, False, mark, config))
        return value
    else:
        value = np.Inf
        for col in valid_moves:
            child = drop_piece(node, col, 3-mark, config)
            value = min(value, minimax_naive(child, depth-1, True, mark, config))
        return value
    
# Uses minimax to calculate value of dropping piece in selected column
def score_n_moves(grid, col, mark, config, nsteps):
    next_grid = drop_piece(grid, col, mark, config)
    score = minimax_naive(next_grid, nsteps-1, False, mark, config) 
    return score

    
def naive_minimax_agent(obs, config):
    
    N_STEPS = 2

    valid_moves = [c for c in range(config.columns) if obs.board[c] == 0]

    grid = np.asarray(obs.board).reshape(config.rows, config.columns)

    scores = dict(zip(valid_moves, [score_n_moves(grid, col, obs.mark, config, N_STEPS) for col in valid_moves]))

    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]

    return random.choice(max_cols)

### Mimimax accelerated

In [125]:
def minimax_accelerated(node, depth, maximizingPlayer, mark, config):
    is_terminal = is_terminal_node(node, config)
    valid_moves = [c for c in range(config.columns) if node[0][c] == 0]
    if depth == 0 or is_terminal:
        return 0
    if maximizingPlayer:
        value = -np.Inf 
        for col in valid_moves:
            child = drop_piece(node, col, mark, config)
            change = change_in_heuristic(node, heuristic_3_window, col, mark, config, maximizingPlayer)
            value = max(value, minimax_accelerated(child, depth-1, False, mark, config) + change)
        return value
    else:
        value = np.Inf
        for col in valid_moves:
            child = drop_piece(node, col, 3-mark, config)
            change = change_in_heuristic(node, heuristic_3_window, col, 3-mark, config, maximizingPlayer)
            value = min(value, minimax_accelerated(child, depth-1, True, mark, config) + change)
        return value

def score_n_moves_accel(grid, col, mark, config, nsteps):
    next_grid = drop_piece(grid, col, mark, config)
    score = minimax_accelerated(next_grid, nsteps-1, False, mark, config) + change_in_heuristic(grid, heuristic_3_window, col, mark, config, True)
    return score    
    
def accelerated_minimax_agent(obs, config):
    
    N_STEPS = 3

    valid_moves = [c for c in range(config.columns) if obs.board[c] == 0]

    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    
    scores = dict(zip(valid_moves, [score_n_moves_accel(grid, col, obs.mark, config, N_STEPS) for col in valid_moves]))

    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]

    return random.choice(max_cols)

### Minimax with prunning

In [126]:
def minimax_with_prunning(node, depth, maximizingPlayer, mark, cutoff, config):
    is_terminal = is_terminal_node(node, config)
    valid_moves = [c for c in range(config.columns) if node[0][c] == 0]
    if depth == 0 or is_terminal:
        return 0
    if maximizingPlayer:
        value = -np.Inf 
        for col in valid_moves:
            child = drop_piece(node, col, mark, config)
            change = change_in_heuristic(node, heuristic_3_window, col, mark, config, maximizingPlayer)
            value = max(value, minimax_with_prunning(child, depth-1, False, mark, -np.Inf, config) + change)
        return value
    else:
        value = np.Inf
        for col in valid_moves:
            child = drop_piece(node, col, 3-mark, config)
            change = change_in_heuristic(node, heuristic_3_window, col, 3-mark, config, maximizingPlayer)
            value = min(value, minimax_with_prunning(child, depth-1, True, mark, -np.Inf, config) + change)
            if value < cutoff:
                break
        return value

def score_n_moves_prun(grid, col, mark, cutoff, nsteps, config):
    next_grid = drop_piece(grid, col, mark, config)
    move_value = change_in_heuristic(grid, heuristic_3_window, col, mark, config, True)
    score = minimax_with_prunning(next_grid, nsteps-1, False, mark, cutoff - move_value, config) + move_value
    return score        
    
def minimax_with_prunning_agent(obs, config):
    
    N_STEPS = 3

    valid_moves = [c for c in range(config.columns) if obs.board[c] == 0]

    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    
    best_score = -np.Inf
    
    for move in valid_moves:
        score = score_n_moves_prun(grid, move, obs.mark, best_score, N_STEPS, config)
        if score > best_score:
            best_score = score
            best_moves = [move]
        elif score == best_score:
            best_moves.append(move)
    
    return random.choice(best_moves)


def deep_minimax_agent(obs, config):
    
    import time
    start_time = time.time()
    
    N_STEPS = 4

    valid_moves = [c for c in range(config.columns) if obs.board[c] == 0]

    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    
    best_score = -np.Inf
    
    for move in valid_moves:
        score = score_n_moves_prun(grid, move, obs.mark, best_score, N_STEPS, config)
        if score > best_score:
            best_score = score
            best_moves = [move]
        elif score == best_score:
            best_moves.append(move)
            
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time} seconds, for deep_minimax agent")
    
    return random.choice(best_moves)

### Sumbissions for competition

In [2]:
def my_agent(obs, config):
    
    import random
    import numpy as np
    
    def drop_piece(grid, col, piece, config):
        next_grid = grid.copy()
        for row in range(5, -1, -1):
            if next_grid[row][col] == 0:
                break
        next_grid[row][col] = piece
        return next_grid
    
    def check_window(window, num_discs, piece, config):
        return (window.count(piece) == num_discs and window.count(0) == 4-num_discs)
    
    def heuristic_3_window(window, piece, config):
        return (
        500 * check_window(window, 4, piece, config)
        + 10 * check_window(window, 3, piece, config)
        + 2 * check_window(window, 2, piece, config)
        - check_window(window, 2, 3 - piece, config)
        - 10 * check_window(window, 3, 3 - piece, config)
        - 1000 * check_window(window, 4, 3 - piece, config)
    )
    
    def change_in_heuristic(grid, heuristic, col, piece, config, maximizingPlayer):
        change = 0
        for row in range(5, -1, -1):
            if grid[row][col] == 0:
                break  
        # horizontal
        for i in range(4):
            if col - i >= 0 and col - i <= 3:
                old_window = list(grid[row, col-i:col+4-i]) 
                new_window = old_window[:]
                new_window[i] = piece
                if maximizingPlayer:
                    change += heuristic(new_window, piece, config) - heuristic(old_window, piece, config)
                else:
                    change += heuristic(new_window, 3-piece, config) - heuristic(old_window, 3-piece, config)
     
        # vertical      
        for i in range(4):
            if row - i >= 0 and row - i <= 2:
                old_window = list(grid[row-i:row-i+4, col]) 
                new_window = old_window[:]
                new_window[i] = piece
                if maximizingPlayer:
                    change += heuristic(new_window, piece, config) - heuristic(old_window, piece, config)
                else:
                    change += heuristic(new_window, 3-piece, config) - heuristic(old_window, 3-piece, config)
     
        # positive diagonal
        for i in range(4):
            if col >= i and col <= 3+i and row >= 3-i and row <= 5-i:
                old_window = list(grid[np.arange(row+i,row+i-4,-1), np.arange(col-i,col+4-i)])
                new_window = old_window[:]   
                new_window[i] = piece
                if maximizingPlayer:
                    change += heuristic(new_window, piece, config) - heuristic(old_window, piece, config)
                else:
                    change += heuristic(new_window, 3-piece, config) - heuristic(old_window, 3-piece, config)
     
        # negative diagonal
        for i in range(4):
            if col-i >= 0 and col-i <= 3 and row-i >= 0 and row-i <= 2:
                old_window = list(grid[np.arange(row-i,row-i+4), np.arange(col-i,col-i+4)])
                new_window = old_window[:] 
                new_window[i] = piece   
                if maximizingPlayer:
                    change += heuristic(new_window, piece, config) - heuristic(old_window, piece, config)
                else:
                    change += heuristic(new_window, 3-piece, config) - heuristic(old_window, 3-piece, config)
     
        return change
    
    
    def is_terminal_window(window, config):
        return window.count(1) == 4 or window.count(2) == 4
    
    def is_terminal_node(grid, config):
        # Check for draw 
        if list(grid[0, :]).count(0) == 0:
            return True
        # Check for win: horizontal, vertical, or diagonal
        # horizontal 
        for row in range(6):
            for col in range(4):
                window = list(grid[row, col:col+4])
                if is_terminal_window(window, config):
                    return True
        # vertical
        for row in range(3):
            for col in range(7):
                window = list(grid[row:row+4, col])
                if is_terminal_window(window, config):
                    return True
        # positive diagonal
        for row in range(3):
            for col in range(4):
                window = list(grid[np.arange(row,row+4), np.arange(col,col+4)])
                if is_terminal_window(window, config):
                    return True
        # negative diagonal
        for row in range(3, 6):
            for col in range(4):
                window = list(grid[np.arange(row, row-4, -1), np.arange(col,col+4)])
                if is_terminal_window(window, config):
                    return True
        return False
    
    def minimax_with_prunning(node, depth, maximizingPlayer, mark, cutoff, config):
        is_terminal = is_terminal_node(node, config)
        valid_moves = [c for c in range(7) if node[0][c] == 0]
        if depth == 0 or is_terminal:
            return 0
        if maximizingPlayer:
            value = -np.Inf 
            for col in valid_moves:
                child = drop_piece(node, col, mark, config)
                change = change_in_heuristic(node, heuristic_3_window, col, mark, config, maximizingPlayer)
                value = max(value, minimax_with_prunning(child, depth-1, False, mark, -np.Inf,  config) + change)
            return value
        else:
            value = np.Inf
            for col in valid_moves:
                child = drop_piece(node, col, 3-mark, config)
                change = change_in_heuristic(node, heuristic_3_window, col, 3-mark, config, maximizingPlayer)
                value = min(value, minimax_with_prunning(child, depth-1, True, mark, -np.Inf, config) + change)
                if value < cutoff:
                    break
            return value
        
    def score_n_moves_prun(grid, col, mark, cutoff, nsteps, config):
        next_grid = drop_piece(grid, col, mark, config)
        move_value = change_in_heuristic(grid, heuristic_3_window, col, mark, config, True)
        score = minimax_with_prunning(next_grid, nsteps-1, False, mark, cutoff - move_value, config) + move_value
        return score
    
    N_STEPS = 4

    valid_moves = [c for c in range(7) if obs.board[c] == 0]

    grid = np.asarray(obs.board).reshape(6, 7)
    
    best_score = -np.Inf
    
    for move in valid_moves:
        score = score_n_moves_prun(grid, move, obs.mark, best_score, N_STEPS, config)
        if score > best_score:
            best_score = score
            best_moves = [move]
        elif score == best_score:
            best_moves.append(move)
   
    return random.choice(best_moves)

### Best submission 

In [128]:
def my_agents(obs, config):
    
    import random
    import numpy as np
    
    def drop_piece(grid, col, piece):
        next_grid = grid.copy()
        for row in range(5, -1, -1):
            if next_grid[row][col] == 0:
                break
        next_grid[row][col] = piece
        return next_grid
    
    def check_window(window, num_discs, piece):
        return (window.count(piece) == num_discs and window.count(0) == 4-num_discs)
    
    def heuristic_3_window(window, piece):
        return (
        1000 * check_window(window, 4, piece)
        + 100 * check_window(window, 3, piece)
        + 20 * check_window(window, 2, piece)
        - 10*check_window(window, 2, 3 - piece)
        - 100 * check_window(window, 3, 3 - piece)
        - 1000 * check_window(window, 4, 3 - piece)
    )
    
    def change_in_heuristic(grid, heuristic, col, piece, maximizingPlayer):
        change = 0
        for row in range(5, -1, -1):
            if grid[row][col] == 0:
                break  
        # horizontal
        for i in range(4):
            if col - i >= 0 and col - i <= 3:
                old_window = list(grid[row, col-i:col+4-i]) 
                new_window = old_window[:]
                new_window[i] = piece
                if maximizingPlayer:
                    change += (0.8+0.15*row)*(heuristic(new_window, piece) - heuristic(old_window, piece))
                else:
                    change += (0.8+0.15*row)*(heuristic(new_window, 3-piece) - heuristic(old_window, 3-piece))
     
        # vertical      
        for i in range(4):
            if row - i >= 0 and row - i <= 2:
                old_window = list(grid[row-i:row-i+4, col]) 
                new_window = old_window[:]
                new_window[i] = piece
                if maximizingPlayer:
                    change += (heuristic(new_window, piece) - heuristic(old_window, piece))
                else:
                    change += (heuristic(new_window, 3-piece) - heuristic(old_window, 3-piece))
     
        # negative diagonal
        for i in range(4):
            if col >= i and col <= 3+i and row >= 3-i and row <= 5-i:
                old_window = list(grid[np.arange(row+i,row+i-4,-1), np.arange(col-i,col+4-i)])
                new_window = old_window[:]   
                new_window[i] = piece
                if maximizingPlayer:
                    change += (0.5+0.15*(i+row))*(heuristic(new_window, piece) - heuristic(old_window, piece))
                else:
                    change += (0.5+0.15*(i+row))*(heuristic(new_window, 3-piece) - heuristic(old_window, 3-piece))
     
        # positive diagonal
        for i in range(4):
            if col-i >= 0 and col-i <= 3 and row-i >= 0 and row-i <= 2:
                old_window = list(grid[np.arange(row-i,row-i+4), np.arange(col-i,col-i+4)])
                new_window = old_window[:] 
                new_window[i] = piece   
                if maximizingPlayer:
                    change += (1.25-0.15*(i-row))*(heuristic(new_window, piece) - heuristic(old_window, piece))
                else:
                    change += (1.25-0.15*(i-row))*(heuristic(new_window, 3-piece) - heuristic(old_window, 3-piece))
     
        return change
    
    
    def is_terminal_window(window):
        return window.count(1) == 4 or window.count(2) == 4
    
    def is_terminal_node(grid):
        # Check for draw 
        if list(grid[0, :]).count(0) == 0:
            return True
        # Check for win: horizontal, vertical, or diagonal
        # horizontal 
        for row in range(6):
            for col in range(4):
                window = list(grid[row, col:col+4])
                if is_terminal_window(window):
                    return True
        # vertical
        for row in range(3):
            for col in range(7):
                window = list(grid[row:row+4, col])
                if is_terminal_window(window,):
                    return True
        # positive diagonal
        for row in range(3):
            for col in range(4):
                window = list(grid[np.arange(row,row+4), np.arange(col,col+4)])
                if is_terminal_window(window):
                    return True
        # negative diagonal
        for row in range(3, 6):
            for col in range(4):
                window = list(grid[np.arange(row, row-4, -1), np.arange(col,col+4)])
                if is_terminal_window(window):
                    return True
        return False
    
    def minimax_with_prunning(node, depth, maximizingPlayer, mark, cutoff):
        is_terminal = is_terminal_node(node)
        valid_moves = [c for c in range(7) if node[0][c] == 0]
        if depth == 0 or is_terminal:
            return 0
        if maximizingPlayer:
            value = -np.Inf 
            for col in valid_moves:
                child = drop_piece(node, col, mark)
                change = change_in_heuristic(node, heuristic_3_window, col, mark, maximizingPlayer)
                value = max(value, minimax_with_prunning(child, depth-1, False, mark, -np.Inf) + change)
            return value
        else:
            value = np.Inf
            for col in valid_moves:
                child = drop_piece(node, col, 3-mark)
                change = change_in_heuristic(node, heuristic_3_window, col, 3-mark, maximizingPlayer)
                value = min(value, minimax_with_prunnins(child, depth-1, True, mark, -np.Inf) + change)
                if value < cutoff:
                    break
            return value
        
    def score_n_moves_prun(grid, col, mark, cutoff, nsteps):
        next_grid = drop_piece(grid, col, mark)
        move_value = change_in_heuristic(grid, heuristic_3_window, col, mark, True)
        score = minimax_with_prunning(next_grid, nsteps-1, False, mark, cutoff - move_value) + move_value
        return score
    
    valid_moves = [c for c in range(7) if obs.board[c] == 0]

    grid = np.asarray(obs.board).reshape(6, 7)
    
    best_score = -np.Inf
    
    for move in valid_moves:
        score = score_n_moves_prun(grid, move, obs.mark, best_score, 4)
        if score > best_score:
            best_score = score
            best_moves = [move]
        elif score == best_score:
            best_moves.append(move)
    
    return random.choice(best_moves)

### Test games

In [129]:
env = make("connectx", debug=True)
# print(list(env.agents))

Manual agents game

get_win_percentages(manual_agent_improved, manual_agent, 50)

Comparing speed of basic heuristic agents

get_win_percentages(base_heuristic_agent, base_heuristic_agent, 40)
get_win_percentages(fast_heuristic_agent, fast_heuristic_agent, 40)

Base vs fast

get_win_percentages(base_heuristic_agent, fast_heuristic_agent, 40)

Fast heuristic vs manual_improved

get_win_percentages(fast_heuristic_agent, manual_agent_improved, 20)

Against negamax

get_win_percentages(fast_heuristic_agent, 'negamax', 10)

Heuristics Battle

get_win_percentages(fast_heuristic_agent, base_heuristic_agent, 20)

 get_win_percentages(my_agent, my_agents, 40)

In [130]:
env.run([deep_minimax_agent, my_agents])
env.render(mode="ipython", width=500, height=450)

Elapsed time: 1.185520887374878 seconds, for deep_minimax agent
Elapsed time: 0.64306640625 seconds, for my agent
Elapsed time: 0.9773702621459961 seconds, for deep_minimax agent
Elapsed time: 0.4491558074951172 seconds, for my agent
Elapsed time: 0.7897753715515137 seconds, for deep_minimax agent
Elapsed time: 0.5862019062042236 seconds, for my agent
Elapsed time: 1.093705177307129 seconds, for deep_minimax agent
Elapsed time: 0.44783926010131836 seconds, for my agent
Elapsed time: 0.9803860187530518 seconds, for deep_minimax agent
Elapsed time: 0.589608907699585 seconds, for my agent
Elapsed time: 0.8523480892181396 seconds, for deep_minimax agent
Elapsed time: 0.588564395904541 seconds, for my agent
Elapsed time: 1.197319746017456 seconds, for deep_minimax agent
Elapsed time: 0.6522858142852783 seconds, for my agent
Elapsed time: 1.0840389728546143 seconds, for deep_minimax agent
Elapsed time: 0.5251891613006592 seconds, for my agent
Elapsed time: 1.164191722869873 seconds, for deep